# Bài toán
**Xây dựng thuật toán dự đoán giá bán của một sản phẩm dựa trên bộ dữ liệu cho trước của Mercari bao gồm các thông tin như danh mục sản phẩm, thương hiệu, mô tả mặt hàng.**

Import thư viện cho bài toán

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = 12,6
import warnings
warnings.filterwarnings('ignore')
seq_col_brew = sns.color_palette("YlGnBu_r", 4)
sns.set_palette(seq_col_brew)

In [ ]:
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Giải nén dữ liệu train

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z

In [ ]:
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

Đọc dữ liệu

In [ ]:
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test_stg2.tsv', sep='\t')

Quan sát dữ liệu ở 2 tập train và test

In [ ]:
train.head()

In [ ]:
test.head()

* Mỗi hàng ở bảng biểu diễn một sản phẩm với các thuộc tính name, item_condition_id, category_name, shipping, item_description. 
* Ở tập train có tập price mà tập test không có vậy nên ta sẽ phải dự đoán giá của sản phẩm ở tập test này.
* Cột brand_name có nhiều giá trị NaN (giá trị null)
* Một sản phẩm có thể thuộc nhiều thể loại (có nhiều giá trị ở cột category_name)

In [ ]:
# Xem thông tin chi tiết về kiểu dữ liệu trong tập train
train.info()

In [ ]:
# Xem thông tin chi tiết về kiểu dữ liệu trong tập test
test.info()

In [ ]:
# Đếm các giá trị NaN trong tập train
pd.isnull(train).sum()

In [ ]:
# Đếm các giá trị NaN trong tập test
pd.isnull(test).sum()

Ta sẽ thay thể các giá trị NaN (giá trị rỗng) để đánh giá giản phẩm chính xác hơn

In [ ]:
# Thay thế các giá trị NaN trong cột brand_name thành Other
train['brand_name'].fillna('Other', inplace=True)
test['brand_name'].fillna('Other', inplace=True)

In [ ]:
# Thay thế các giá trị NaN trong cột category_name thành No/No/No
train['category_name'].fillna('No/No/No', inplace=True)
test['category_name'].fillna('No/No/No', inplace=True)

In [ ]:
# Thay thế các giá trị NaN trong cột item_description thành No description yet
train['item_description'].fillna('No description yet', inplace=True)
test['item_description'].fillna('No description yet', inplace=True)

In [ ]:
pd.isnull(train).sum()

In [ ]:
pd.isnull(test).sum()

In [ ]:
#Quan sát dữ liệu ở tập train sau khi thay thế các giá trị NaN
train.head()

In [ ]:
#Quan sát dữ liệu ở tập test sau khi thay thế các giá trị NaN
test.head()

In [ ]:
#kiểm tra category_name
levels =[]

for values in train['category_name']:
    levels.append(values.count("/"))


print("MIN  of levels:          {0:.0f} ".format(np.min(levels)+1))
print("MEDIAN of levels:        {0:.0f} ".format(np.percentile(levels, 50)+1))
print("MAX  of levels:          {0:.0f}".format(np.max(levels)+1))
print("90 percentile  of levels:{0:.0f} ".format(np.percentile(levels, 90)+1))

Cột category_name có các mục được phân tách bởi dấu "/", tối đa là 5 và tối thiểu là 3
Phần lớn sản phẩm chỉ có 3 mục nên ta sẽ chia cột category thành 3 cột khác nhau giúp đánh giá sản phẩm dễ hơn


In [ ]:
#Hàm split_cat dùng để cắt các nhãn trong cột 'category_name' thành các cột riêng biệt
def split_cat(category_name):
    try:
        return category_name.split('/')
        return ("done")
    except:
        return ['Null', 'Null', 'Null']

In [ ]:
train['category_1'], train['category_2'], train['category_3'] = zip(*train['category_name'].apply(lambda x:split_cat(x)))
test['category_1'], test['category_2'], test['category_3'] = zip(*test['category_name'].apply(lambda x: split_cat(x)))

In [ ]:
train.head()

In [ ]:
test.head()

Tiếp theo ta quan tâm tới các cột chứa dữ liệu dạng số

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
train.describe()

Biểu đồ phân bố giá của các sản phẩm trong tập train

In [ ]:
# Biểu đồ phân bố giá của các sản phẩm trong tập train
plt.figure(figsize=(10,5))
plt.hist(train['price'], bins = 100, edgecolor='white', range=[0,250])
plt.ylabel('Frequency')
plt.xlabel('Price')
plt.show()

Chúng ta có thể thấy phân phối giá sản phẩm bị lệch về bên trái nên ta sẽ đưa về phân phối chuẩn với logarit price

In [ ]:
#Biểu đồ phân phối logarit price
plt.figure(figsize=(10,5))
plt.hist(np.log(train['price']+1),bins=50, edgecolor='white') 
plt.ylabel('Frequency')
plt.xlabel('Log_price')
plt.show()

Phân bố của logarit price phân bố xung quanh giá trị trung bình nên ta sẽ sử dụng logarit price để huấn luyện

In [ ]:
#Biểu đồ phân tích cột item_condition_id
ax = sns.countplot(x = 'item_condition_id',data=train, palette ='Blues_r')
ax.set_title("Tổng số lượng sản phẩm theo item_condition_id", fontsize = 13)

Có 5 loại item_condition_id
Trong đó loại 1,2,3 phổ biến; loại 4,5 không phổ biến

In [ ]:
# Phí vận chuyển
(train['shipping'].value_counts())*100/train.shape[0]

Phí vận chuyển được trả bởi người bán là 0, người mua là 1

In [ ]:
plt.figure(figsize=(10,8))

plt.hist(np.log(train['price'][train['shipping']==0]+1),bins=50, edgecolor='white',color="blue", label='shipping = 0') 
plt.hist(np.log(train['price'][train['shipping']==1]+1),bins=50, edgecolor='white',color="cornflowerblue",label="shipping = 1") 
plt.xlabel('Log_price')
plt.legend(loc='upper right')
plt.title('Shipping')
plt.show()

Giá trị trung bình của những sản phẩm có shipping = 0 (người mua không mất phí ship) nhỏ hơn giá trị trung bình của những sản phẩm có shipping = 1 (người mua phải trả thêm tiền ship)

In [ ]:
#danh sách 10 brand phổ biến nhất sắp xếp giảm dần
brands = train['brand_name'].value_counts()
print(brands[:10])

In [ ]:
#Quan sát phân phối giá của các thương hiệu phổ biến nhất
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 15))
plt.subplot(3, 3, 1)#hình có 3 hàng 3 cột và đây là biểu đò đầu tiên trong hình
plt.hist(np.log(train['price'][train['brand_name']=='Other']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('Nobrand')

plt.subplot(3, 3, 2)
plt.hist(np.log(train['price'][train['brand_name']=='Nike']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('Nike')

plt.subplot(3, 3, 3)
plt.hist(np.log(train['price'][train['brand_name']=="Victoria's Secret"]+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title("Victoria's Secret")

plt.subplot(3, 3, 4)
plt.hist(np.log(train['price'][train['brand_name']=='LuLaRoe']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('LuLaRoe')

plt.subplot(3, 3, 5)
plt.hist(np.log(train['price'][train['brand_name']=='Apple']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('Apple')

plt.subplot(3, 3, 6)
plt.hist(np.log(train['price'][train['brand_name']=='FOREVER 21']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('FOREVER 21')

plt.subplot(3, 3, 7)
plt.hist(np.log(train['price'][train['brand_name']=='Nintendo']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('Nintendo')

plt.subplot(3, 3, 8)
plt.hist(np.log(train['price'][train['brand_name']=='Lululemon']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('Lululemon')

plt.subplot(3, 3, 9)
plt.hist(np.log(train['price'][train['brand_name']=='Michael Kors']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('Michael Kors')

plt.show()

In [ ]:
#10 brand ít phổ biến hơn
print(brands[250:260])

In [ ]:
plt.figure(figsize=(20, 15))

plt.subplot(3, 3, 1)
plt.hist(np.log(train['price'][train['brand_name']=='Sperrys']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('Sperrys')

plt.subplot(3, 3, 2)
plt.hist(np.log(train['price'][train['brand_name']=='Disney Collection']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('Disney Collection')

plt.subplot(3, 3, 3)
plt.hist(np.log(train['price'][train['brand_name']=='Madewell']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('Madewell')

plt.subplot(3, 3, 4)
plt.hist(np.log(train['price'][train['brand_name']=='Mudd']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('Mudd')

plt.subplot(3, 3, 5)
plt.hist(np.log(train['price'][train['brand_name']=='7 For All Mankind®']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('7 For All Mankind®')

plt.subplot(3, 3, 6)
plt.hist(np.log(train['price'][train['brand_name']=='Gerber']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('Gerber')

plt.subplot(3, 3, 7)
plt.hist(np.log(train['price'][train['brand_name']=='Diamond Supply Co.']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('Diamond Supply Co.')

plt.subplot(3, 3, 8)
plt.hist(np.log(train['price'][train['brand_name']=='Leap Frog']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('Leap Frog')

plt.subplot(3, 3, 9)
plt.hist(np.log(train['price'][train['brand_name']=='BCBGMAXAZRIA']+1),bins=50, edgecolor='white')
plt.xlabel('Log_price')
plt.ylabel('Frequency')
plt.title('BCBGMAXAZRIA')

plt.show()

Hầu hết các phân phối Logarit price của các hãng đều tuân theo phân phối chuẩn

In [ ]:
print("Có %d nhãn ở cột category_1." % train['category_1'].nunique())

In [ ]:
plt.figure(figsize=(10,8))
ax = sns.countplot(x = 'category_1', data = train, palette ='Blues_r', edgecolor='k')
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
ax.set_title('Biểu đồ số lượng các sản phẩm chứa các nhãn ở cột category_1')

Bảng phân bố giá của sản phẩm ứng với từng loại trong category_1

In [ ]:
train.groupby('category_1')['price'].describe()

In [ ]:
print("Có %d nhãn ở cột category_2." % train['category_2'].nunique())

In [ ]:
plt.figure(figsize=(10,8))
ax = sns.countplot(x='category_2',data=train,order=train.category_2.value_counts().iloc[:20].index, palette ='Reds_r', edgecolor='k')
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
ax.set_title('Biểu đồ số lượng sản phẩm chứa các nhãn ở category_2 ')

Bảng phân bố giá của sản phẩm ứng với từng loại trong category_2

In [ ]:
train.groupby('category_2')['price'].describe()[:10]

In [ ]:
print("Có %d nhãn ở cột category_3." % train['category_3'].nunique())

In [ ]:
plt.figure(figsize=(10,8))
ax = sns.countplot(x='category_3',data=train,order=train.category_3.value_counts().iloc[:20].index, palette ='Greens_r', edgecolor='k')
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
ax.set_title('Biểu đồ số lượng sản phẩm chứa các nhãn ở category_3 ')

Bảng phân bố giá của sản phẩm ứng với từng loại trong category_3

In [ ]:
train.groupby('category_3')['price'].describe()[:10]

In [ ]:
# Đếm số lượng 1 số từ trong 'name'
train['name'].value_counts()[:10]

In [ ]:
x = train['name'].apply(lambda x: len(x))
plt.hist(x,bins = 30,range=[0,50],edgecolor='white')
plt.show()

Quan sát dữ liệu ở cột item_description

In [ ]:
from wordcloud import WordCloud
import os
wordcloud = WordCloud(width = 2400, height = 1200).generate(" ".join(train.item_description.astype(str)))
plt.figure(figsize = (13, 10))
plt.imshow(wordcloud)
plt.show()

Các mô tả "free shipping", "great condition", "Brand new", "description yet" là những mô tả được sử dụng nhiều nhất

**Tiếp theo để huấn luyện mô hình ta cần đưa dữ liệu về dạng vector để huấn luyện theo mô hình hồi quy tuyến tính**

Sử dụng LabelBinarizer để chuyển đổi các nhãn nhiều lớp sang nhãn nhị phân

In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb_item_condition_id = LabelBinarizer(sparse_output=True)
train_condition = lb_item_condition_id.fit_transform(train['item_condition_id'])
test_condition = lb_item_condition_id.transform(test['item_condition_id'])

In [ ]:
train_condition.shape

Có 1482535 sản phẩm với kiểu item_condition

In [ ]:
lb_shipping = LabelBinarizer(sparse_output=True)
train_shipping = lb_shipping.fit_transform(train['shipping'])
test_shipping = lb_shipping.transform(test['shipping'])

In [ ]:
train_shipping.shape

Có 1482535 sản phẩm với kiểu item_condition với 2 kiểu shipping

In [ ]:
lb_brand_name = LabelBinarizer(sparse_output=True)
train_brand_name= lb_brand_name.fit_transform(train['brand_name'])
test_brand_name = lb_brand_name.transform(test['brand_name'])

In [ ]:
train_brand_name.shape

Có 1482535 sản phẩm với 4810 brand

Ta sử dụng hàm CountVectorizer() để chuyển cột name từ dạng text về dạng một vecto trên cơ sở số lần xuất hiện của mỗi từ

CountVectorizer tạo một ma trận trong đó mỗi từ duy nhất được biểu thị bằng một cột của ma trận và với tên của mỗi sản phẩm là một hàng trong ma trận. Giao của một hàng và một cột chính là số lần xuất hiện của từ(tương úng với cột) trong tên của sản phẩm (tương úng với hàng)

In [ ]:
count_vec = CountVectorizer()

train_name = count_vec.fit_transform(train['name'])
test_name = count_vec.transform(test['name'])

In [ ]:
print(train_name.shape)

Có 1482535 sản phẩm với 105757 từ xuất hiện trong cột name

Ta sử dụng hàm TfidfVectorizer() để chuyển đổi dữ liệu ở cột item_descreption thành ma trận

In [ ]:
tfidf_des = TfidfVectorizer(max_features=50000, ngram_range=(1, 3), stop_words='english')

train_des = tfidf_des.fit_transform(train['item_description'])
test_des = tfidf_des.transform(test['item_description'])

In [ ]:
train_des.shape

Sau khi chuẩn hóa dữ liệu ở các cột category_name, sử dụng LabelBinarizer để chuyển đổi các nhãn nhiều lớp sang nhãn nhị phân ở các cột category_1, category_2, category_3

In [ ]:
lb_cat_1 = LabelBinarizer(sparse_output=True)
train_cat_1 = lb_cat_1.fit_transform(train['category_1'])
test_cat_1 = lb_cat_1.transform(test['category_1'])

lb_cat_2 = LabelBinarizer(sparse_output=True)
train_cat_2 = lb_cat_2.fit_transform(train['category_2'])
test_cat_2 = lb_cat_2.transform(test['category_2'])

lb_cat_3 = LabelBinarizer(sparse_output=True)
train_cat_3 = lb_cat_3.fit_transform(train['category_3'])
test_cat_3 = lb_cat_3.transform(test['category_3'])

Tạo sparse matrix để kết hợp các dữ liệu với nhau

In [ ]:
from scipy.sparse import hstack
import gc

In [ ]:
sparse_matrix_list = (train_name, train_des, train_brand_name, train_condition,
                      train_shipping, train_cat_1, train_cat_2, train_cat_3)
X_train = hstack(sparse_matrix_list).tocsr()
print( X_train.shape)

del X_train
gc.collect()

# Dự đoán giá trên tập test

In [ ]:
sparse_matrix_list = (train_name, train_des, train_brand_name, train_condition,
                      train_shipping, train_cat_1, train_cat_2, train_cat_3)
X_train = hstack(sparse_matrix_list).tocsr()
X_train

In [ ]:
y_train = np.log1p(train['price'])
y_train

In [ ]:
Ridge_model =  Ridge()
Ridge_model.fit(X_train, y_train)

In [ ]:
sparse_matrix_list = (test_name, test_des, test_brand_name, test_condition,
                      test_shipping, test_cat_1, test_cat_2, test_cat_3)
X_test = hstack(sparse_matrix_list).tocsr()

In [ ]:
preds = Ridge_model.predict(X_test)
preds

In [ ]:
preds = np.expm1(preds)
preds

In [ ]:
submission = pd.read_csv('sample_submission_stg2.csv')
submission

In [ ]:
submission.loc[:, 'price'] = preds
submission

In [ ]:
# chuyển submission_df sang dạng .csv để submit
submission.to_csv('submission.csv', index=False)